# **Data Cleaning: Process Each Source Individually**
**Run after:** data_exploration.ipynb.  
**Outputs:** Cleaned CSVs in data/processed/ (accidents, mobility, weather, roads).  


## **Imports & Paths**

In [2]:
# Imports
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import from_wkt
import openpyxl  # XLSX
import warnings
import os
warnings.filterwarnings("ignore")

# Paths (relative to root from src/)
raw_path = "../data/raw/"
processed_path = "../data/processed/"
os.makedirs(processed_path, exist_ok=True)

print("=== Data Cleaning Started ===")
print(f"Raw path: {raw_path} (files: {os.listdir(raw_path)})")
print(f"Processed path: {processed_path} (created if missing)")

=== Data Cleaning Started ===
Raw path: ../data/raw/ (files: ['2020_PK_Region_Mobility_Report.xlsx', '2021_PK_Region_Mobility_Report.xlsx', '2022_PK_Region_Mobility_Report.xlsx', 'hotosm_pak_roads_lines_shp.dbf', 'hotosm_pak_roads_lines_shp.shp', 'hotosm_pak_roads_lines_shp.shx', 'pakistan_weather_2000_2024.csv', 'pakistan_weather_data-Sep2024-Oct2025.csv', 'RTA Data 2020 to July 2023.xlsx'])
Processed path: ../data/processed/ (created if missing)


## **RTA Cleaning**
- Drop missing CallTime (13%).
- Parse datetime, filter 2020-2023.
- incident_count = TotalPatientsInEmergency.fillna(1).
- severity = PatientStatus.fillna('Unknown').
- Proxy geo: Lahore center + jitter ±0.02° (~2km variance).
- Filter Lahore bbox.
- Output: cleaned_accidents.csv (shape ~40k).

In [3]:
print("\n=== Cleaning RTA Accidents ===")
rta_path = raw_path + "RTA Data 2020 to July 2023.xlsx"
rta = pd.read_excel(rta_path)

# Basic clean
rta = rta.dropna(subset=['CallTime'])
rta['datetime'] = pd.to_datetime(rta['CallTime'])
rta = rta[rta['datetime'].dt.year.between(2020, 2023)]

# Engineer features
rta['incident_count'] = rta['TotalPatientsInEmergency'].fillna(1)
rta['severity'] = rta['PatientStatus'].fillna('Unknown')

# Lahore geo proxy (center + jitter for variance)
np.random.seed(42)  # Reproducible
base_lat, base_lon = 31.55, 74.35
rta['Latitude'] = base_lat + np.random.normal(0, 0.02, len(rta))  # ~2km N-S jitter
rta['Longitude'] = base_lon + np.random.normal(0, 0.02, len(rta))  # ~2km E-W jitter

# Lahore bbox filter
lahore_bounds = (31.4, 74.1, 31.7, 74.5)  # (min_lat, min_lon, max_lat, max_lon)
rta_lahore = rta[
    (rta['Latitude'].between(lahore_bounds[0], lahore_bounds[2])) &
    (rta['Longitude'].between(lahore_bounds[1], lahore_bounds[3]))
][['datetime', 'Latitude', 'Longitude', 'incident_count', 'severity']]

print(f"Raw shape: {rta.shape}, Lahore shape: {rta_lahore.shape}")
print("Geo stats (post-jitter):\n", rta_lahore[['Latitude', 'Longitude']].describe().round(4))
print("Incident count stats:\n", rta_lahore['incident_count'].describe())

# Save
rta_lahore.to_csv(processed_path + "cleaned_accidents.csv", index=False)
print("✓ Saved cleaned_accidents.csv")


=== Cleaning RTA Accidents ===
Raw shape: (40232, 30), Lahore shape: (40232, 5)
Geo stats (post-jitter):
          Latitude   Longitude
count  40232.0000  40232.0000
mean      31.5500     74.3501
std        0.0200      0.0200
min       31.4607     74.2607
25%       31.5364     74.3366
50%       31.5500     74.3501
75%       31.5635     74.3636
max       31.6396     74.4252
Incident count stats:
 count    40232.000000
mean         1.139068
std          0.479940
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         15.000000
Name: incident_count, dtype: float64
✓ Saved cleaned_accidents.csv


## **Mobility Cleaning**
- Concat 3 XLSXs.
- Filter Punjab, 2020-2023.
- Resample 15-min ffill.
- mobility_proxy = workplaces_percent_change_from_baseline.fillna(0).
- Output: cleaned_mobility.csv (shape ~93k).

In [4]:
print("\n=== Cleaning Mobility ===")
files = [
    raw_path + "2020_PK_Region_Mobility_Report.xlsx",
    raw_path + "2021_PK_Region_Mobility_Report.xlsx",
    raw_path + "2022_PK_Region_Mobility_Report.xlsx"
]
dfs = []
for file in files:
    df = pd.read_excel(file)
    df['datetime'] = pd.to_datetime(df['date'])
    dfs.append(df)
mob_pk = pd.concat(dfs, ignore_index=True)

# Filter
mob_pk = mob_pk[mob_pk['datetime'].dt.year.between(2020, 2023)]
mob_punjab = mob_pk[mob_pk['sub_region_1'] == 'Punjab'][
    ['datetime', 'workplaces_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline']
]

# Resample 15-min ffill
mob_punjab = mob_punjab.set_index('datetime').resample('15min').ffill()
mob_punjab['mobility_proxy'] = mob_punjab['workplaces_percent_change_from_baseline'].fillna(0)

print(f"Raw concat shape: {mob_pk.shape}, Punjab resampled shape: {mob_punjab.shape}")
print("Head:\n", mob_punjab.head(3).to_string())
print("Mobility proxy stats:\n", mob_punjab['mobility_proxy'].describe())

# Save
mob_punjab.reset_index().to_csv(processed_path + "cleaned_mobility.csv", index=False)
print("✓ Saved cleaned_mobility.csv")


=== Cleaning Mobility ===
Raw concat shape: (20404, 16), Punjab resampled shape: (93409, 3)
Head:
                      workplaces_percent_change_from_baseline  transit_stations_percent_change_from_baseline  mobility_proxy
datetime                                                                                                                   
2020-02-15 00:00:00                                      2.0                                            4.0             2.0
2020-02-15 00:15:00                                      2.0                                            4.0             2.0
2020-02-15 00:30:00                                      2.0                                            4.0             2.0
Mobility proxy stats:
 count    93409.000000
mean         8.579077
std         30.337160
min        -79.000000
25%         -9.000000
50%          5.000000
75%         38.000000
max         63.000000
Name: mobility_proxy, dtype: float64
✓ Saved cleaned_mobility.csv


## **Weather Cleaning**
- Concat 2 CSVs.
- Parse datetime, filter Lahore 2020-2023.
- Resample 15-min interpolate.
- precip_lag = precipitation.shift(4).fillna(0) (~1hr lag).
- Output: cleaned_weather.csv (shape ~140k).

In [8]:
print("\n=== Cleaning Weather ===")
w1_path = raw_path + "pakistan_weather_2000_2024.csv"
w2_path = raw_path + "pakistan_weather_data-Sep2024-Oct2025.csv"
w1 = pd.read_csv(w1_path)
w2 = pd.read_csv(w2_path)

# Parse datetime in each
w1['datetime'] = pd.to_datetime(w1['date'], errors='coerce')
w2['datetime'] = pd.to_datetime(w2['date'], errors='coerce')

# Concat
weather = pd.concat([w1, w2], ignore_index=True)

# Re-parse after concat (fixes object dtype)
weather['datetime'] = pd.to_datetime(weather['datetime'], errors='coerce')

# Drop NaT now (after re-parse)
weather = weather.dropna(subset=['datetime'])

print(f"Raw concat shape: {weather.shape}, After NaT drop: {len(weather)}")
print(f"Datetime dtype: {weather['datetime'].dtype}")  # Must be datetime64[ns]

# Lahore filter & year range (.dt safe now)
weather_lahore = weather[
    (weather['city'].str.contains('Lahore', na=False, case=False)) &
    (weather['datetime'].dt.year.between(2020, 2023))
][['datetime', 'tavg', 'prcp', 'humidity', 'latitude', 'longitude']]

# Resample 15-min interpolate
weather_lahore = weather_lahore.set_index('datetime').resample('15T').interpolate(method='linear')
weather_lahore['temperature'] = weather_lahore['tavg']
weather_lahore['precipitation'] = weather_lahore['prcp']
weather_lahore['precip_lag'] = weather_lahore['precipitation'].shift(4).fillna(weather_lahore['precipitation'].iloc[0])

print(f"Lahore raw shape: {len(weather[weather['city'].str.contains('Lahore', na=False, case=False)])}, Resampled shape: {weather_lahore.shape}")
print("Head:\n", weather_lahore.head(3).to_string())
print("Precip stats:\n", weather_lahore['precipitation'].describe().round(2))

# Save
weather_lahore.reset_index().to_csv(processed_path + "cleaned_weather.csv", index=False)
print("✓ Saved cleaned_weather.csv")


=== Cleaning Weather ===
Raw concat shape: (31779, 28), After NaT drop: 31779
Datetime dtype: datetime64[ns]
Lahore raw shape: 3288, Resampled shape: (140161, 8)
Head:
                          tavg  prcp   humidity  latitude  longitude  temperature  precipitation  precip_lag
datetime                                                                                                   
2020-01-01 00:00:00  6.200000   0.0  90.000000   31.5497    74.3436     6.200000            0.0         0.0
2020-01-01 00:15:00  6.234375   0.0  89.947917   31.5497    74.3436     6.234375            0.0         0.0
2020-01-01 00:30:00  6.268750   0.0  89.895833   31.5497    74.3436     6.268750            0.0         0.0
Precip stats:
 count    140161.00
mean          1.60
std           4.14
min           0.00
25%           0.00
50%           0.00
75%           0.74
max          42.60
Name: precipitation, dtype: float64
✓ Saved cleaned_weather.csv


## **Roads Cleaning**
- Load gpd (.shp + .dbf).
- Set CRS EPSG:4326, project to UTM 32643 for length (km).
- Lahore bbox filter.
- Add dummies (segment_id, highway='unknown', name='unnamed').
- Output: cleaned_roads.csv (non-spatial/tabular for merge ; .shp for geospacial joins and visualizations).

In [9]:
print("\n=== Cleaning Roads ===")
os.environ['SHAPE_RESTORE_SHX'] = 'YES'  # SHX index
roads_path = raw_path + "hotosm_pak_roads_lines_shp.shp"
roads_gdf = gpd.read_file(roads_path)
roads_gdf = roads_gdf.set_crs("EPSG:4326", allow_override=True)

# Project for accurate length
roads_projected = roads_gdf.to_crs("EPSG:32643")
roads_projected["length"] = roads_projected.geometry.length / 1000  # km
roads_gdf = roads_projected.to_crs("EPSG:4326")

# Lahore bbox filter
lahore_wkt = 'POLYGON((74.1 31.4, 74.5 31.4, 74.5 31.7, 74.1 31.7, 74.1 31.4))'
lahore_poly = from_wkt(lahore_wkt)
lahore_roads = roads_gdf[roads_gdf.intersects(lahore_poly)].copy()

# Clean invalid/empty
lahore_roads = lahore_roads[lahore_roads.geometry.is_valid & ~lahore_roads.geometry.is_empty]

# Add dummies
lahore_roads["segment_id"] = range(len(lahore_roads))
if 'highway' not in lahore_roads.columns:
    lahore_roads["highway"] = "unknown"
if 'name' not in lahore_roads.columns:
    lahore_roads["name"] = "unnamed"

print(f"Raw shape: {roads_gdf.shape}, Lahore cleaned shape: {lahore_roads.shape}")
print("Length stats:\n", lahore_roads['length'].describe().round(4))
print("Head:\n", lahore_roads.head(3)[['highway', 'name', 'length', 'geometry']].to_string() if 'highway' in lahore_roads.columns else lahore_roads.head(3).to_string())

# Save (CSV for merge, .shp for geo)
lahore_roads[['segment_id', 'highway', 'name', 'length']].to_csv(processed_path + "cleaned_roads.csv", index=False)
lahore_roads.to_file(processed_path + "cleaned_roads.shp")
print("✓ Saved cleaned_roads.csv and .shp")


=== Cleaning Roads ===
Raw shape: (1077644, 16), Lahore cleaned shape: (79354, 17)
Length stats:
 count    79354.0000
mean         0.1781
std          0.3048
min          0.0005
25%          0.0539
50%          0.1038
75%          0.1970
max         20.0204
Name: length, dtype: float64
Head:
          highway         name    length                                                                                                                      geometry
227  residential         None  0.104746                                                                             LINESTRING (74.29559 31.47191, 74.29662 31.47157)
228  residential         None  0.040637                                                                             LINESTRING (74.29684 31.47189, 74.29662 31.47157)
229     tertiary  Ghazan Road  0.038187  LINESTRING (74.29924 31.46942, 74.29917 31.46932, 74.29913 31.46928, 74.2991 31.46923, 74.29907 31.46917, 74.29905 31.46912)
✓ Saved cleaned_roads.csv and .shp


## **Cleaning Summary & Readiness Check**
### Outputs Generated
- **cleaned_accidents.csv**: 40k rows (Lahore jittered incidents, datetime/incident_count/severity/geo).
- **cleaned_mobility.csv**: 93k rows (Punjab 15-min, mobility_proxy filled).
- **cleaned_weather.csv**: 140k rows (Lahore 15-min, precip_lag engineered).
- **cleaned_roads.csv**: 79k rows (Lahore bbox, length/highway dummies; .shp for geo).

### Validation
- All dtypes correct (datetime64[ns], float64 for nums).
- No major data loss (e.g., 13% CallTime drop in RTA handled).
- Geo variance good (std 0.02° for accidents; roads bbox filtered).
- Temporal alignment: 15-min ready for merge.
